<a href="https://colab.research.google.com/github/eric-shaker/BIOENG_1340_AUTO_TUMOR_SEGMENTATION/blob/main/Image_Resample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import the packages
import os
import numpy as np
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm

In [ ]:
# Function to resample the images
def resample_image_standardize(itk_image1, out_size=(64, 64, 64), is_label=False):
    itk_image1.SetOrigin([0, 0, 0])

    original_spacing = itk_image1.GetSpacing()
    original_size = itk_image1.GetSize()

    out_spacing = [
        (original_size[0] * (original_spacing[0] / out_size[0])),
        (original_size[1] * (original_spacing[1] / out_size[1])),
        (original_size[2] * (original_spacing[2] / out_size[2])),
    ]

    reference_image = sitk.Image(out_size, 1)
    reference_image.SetDirection(itk_image1.GetDirection())
    reference_image.SetSpacing(out_spacing)
    reference_image.SetPixel
    interpolator = sitk.sitkNearestNeighbor if is_label else sitk.sitkBSpline

    return sitk.Resample(itk_image1, reference_image, sitk.Transform(), interpolator)

In [ ]:
# Function to rescale the images to a range between 0 and 255
def rescale_image(itk_image2):
    rescaleFilter = sitk.RescaleIntensityImageFilter()
    rescaleFilter.SetOutputMaximum(255)
    rescaleFilter.SetOutputMinimum(0)
    return rescaleFilter.Execute(itk_image2)

# Testing the Functions

In [ ]:
# Get a sample image path
train_path = "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/train_images"
img_path = os.path.join(train_path, os.listdir(train_path)[0])

# Read the image in, downsample it to 64x64x64 voxels, and rescale it between 0 and 255
img = sitk.ReadImage(img_path)
temp = img
img = resample_image_standardize(img, out_size=(32,32,32), is_label=False)
img.SetOrigin(temp.GetOrigin())
sitk.WriteImage(img, "test.nii.gz")
img = rescale_image(img)
sitk.WriteImage(img, "test1.nii.gz")

In [ ]:
# Get a sample seg path
img_path = "/content/gdrive/MyDrive/BIOENG 1340 -Final Project/segs/train_segs_WT/BraTS20_Training_281_seg_WT.nii.gz"

# Read the segmentation in and downsample it to 64x64x64 voxels
img = sitk.ReadImage(img_path)
temp = img
img = resample_image_standardize(img, out_size=(64,64,64), is_label=True)
img.SetOrigin(temp.GetOrigin())
sitk.WriteImage(img, "test_seg.nii.gz")

# Resample all the Images and Segmentations

In [ ]:
os.makedirs("/content/gdrive/MyDrive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images32", exist_ok=True)
os.makedirs("/content/gdrive/MyDrive/BIOENG_1340/BIOENG 1340 - Final Project/images/test_images32", exist_ok=True)
os.makedirs("/content/gdrive/MyDrive/BIOENG_1340/BIOENG 1340 - Final Project/segs/train_segs_WT32", exist_ok=True)
os.makedirs("/content/gdrive/MyDrive/BIOENG_1340/BIOENG 1340 - Final Project/segs/test_segs_WT32", exist_ok=True)

In [ ]:
# Function to input the images into the resampling functions and output them to another directory
def resample_imgs(img_dir, output_path, img_size=(64,64,64), out_name="64"):
  all_img_paths = os.listdir(img_dir)
  for img_name in tqdm(all_img_paths):
    # Read the image in, downsample it to 64x64x64 voxels, and rescale it between 0 and 255
    img = sitk.ReadImage(os.path.join(img_dir,img_name))
    temp = img
    img = resample_image_standardize(img, out_size=img_size, is_label=False)
    img.SetOrigin(temp.GetOrigin())
    img = rescale_image(img)
    idx = img_name.find(".nii")
    output_name = img_name[:idx] + out_name + ".nii.gz"
    sitk.WriteImage(img, os.path.join(output_path, output_name))


In [ ]:
# Function to input the masks into the resampling functions and output them to another directory
def resample_segs(seg_dir, output_path, img_size=(64,64,64), out_name="64"):
  all_seg_paths = os.listdir(seg_dir)
  for seg_name in tqdm(all_seg_paths):
    # Read the mask in and downsample it to 64x64x64 voxels
    seg = sitk.ReadImage(os.path.join(seg_dir,seg_name))
    temp = seg
    seg = resample_image_standardize(seg, out_size=img_size, is_label=True)
    seg.SetOrigin(temp.GetOrigin())
    idx = seg_name.find(".nii")
    output_name = seg_name[:idx] + out_name + seg_name[idx:]
    sitk.WriteImage(seg, os.path.join(output_path, output_name))

In [ ]:
# Resample the images
resample_imgs("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/train_images",
              "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/train_images32",(32,32,32),"32")
resample_imgs("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/test_images",
              "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/test_images32",(32,32,32),"32")

100%|██████████| 50/50 [00:25<00:00,  1.93it/s]


In [ ]:
# Resample the masks
resample_segs("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/segs/train_segs_WT",
              "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/segs/train_segs_WT32",(32,32,32),"32")
resample_segs("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/segs/test_segs_WT",
              "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/segs/test_segs_WT32",(32,32,32),"32")

100%|██████████| 50/50 [00:02<00:00, 24.71it/s]
